## Exercice 1

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F

root_dir = '/home/nesrine/Bureau/DeepLearning/cifar10_data'

#training data
train_data = datasets.CIFAR10(
    root=root_dir, 
    train=True, # we specify train = true to indicate that it is a training a set
    transform=transforms.ToTensor(),
    download=True)
#validation/test data
validation_data = datasets.CIFAR10(
    root=root_dir, 
    train=False, #and here it is a testing set
    transform=transforms.ToTensor(),
    download=True)

labels_dict ={
    0: "plane",
    1: "car",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck"
}


ImportError: /home/nesrine/.local/lib/python3.10/site-packages/torch/lib/libtorch_cuda.so: undefined symbol: ncclCommRegister

In [ ]:
# Affichage des cinq premières images avec leurs étiquettes
for i in range(5):
    image, label = train_data[i]
    print(f"Image{i+1} :")
    print("Shape :",image.shape)
    plt.imshow(image.permute(1, 2, 0))
    plt.title(labels_dict[label])
    plt.axis('on')
    plt.show()

## Exercice 2

In [ ]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, kernel_size=(5,5))
        self.conv2 = nn.Conv2d(6, 16, kernel_size=(5,5))
        self.conv3 = nn.Conv2d(16, 120, kernel_size=(5,5))
        self.fc1 = nn.Linear(120,84)
        self.fc2 = nn.Linear(84,10)

    def forward(self,input):
        layer1 = F.relu(self.conv1(input))
        layer2 = F.max_pool2d(layer1, kernel_size=(2,2), stride=2)
        layer3 = F.relu(self.conv2(layer2))
        layer4 = F.max_pool2d(layer3, kernel_size=(2,2), stride=2)
        layer5 = F.relu(self.conv3(layer4))
        layer6 = F.relu(self.fc1(torch.flatten(layer5,1)))
        layer7 = self.fc2(layer6)
        return layer7


model = LeNet5()
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Number of parameters in model:", num_params)



## Exercice 3

In [ ]:
def accuracy(predictions_proba, targets, top=5):
    tensor_top = torch.topk(predictions_proba, top).indices
    # warning, not yet efficiently implemented
    good_predictions = functorch.vmap(lambda t1, t2: torch.isin(t1,t2))(targets, tensor_top)
    return (good_predictions).sum()/len(targets) * 100

# Plot monitoring
def ith(l, i):
    return [x[i] for x in l]

def plot_history(h):
    fig, axs = plt.subplots(1, 2)
    axs[0].plot(ith(h,0), ith(h,1))
    axs[0].plot(ith(h,0), ith(h,3), color='red')
    axs[0].set_xlabel('epochs')
    axs[0].set_ylabel('loss')
    axs[1].plot(ith(h,0), ith(h,2))
    axs[1].plot(ith(h,0), ith(h,4), color='red')
    axs[1].set_xlabel('epochs')
    axs[1].set_ylabel('accuracy')
    fig.suptitle('Monitor learning')
    fig.tight_layout()
    plt.show()

# subset des données
train_subset = Subset(train_data, range(500))
validation_subset = Subset(validation_data, range(500))

# training and validation loaders for mini batch
train_loader = DataLoader(train_subset, batch_size=256, shuffle=True)
validation_loader = DataLoader(validation_subset, batch_size=256, shuffle = True)

# fonction de perte
CEL = nn.CrossEntropyLoss()

def grad_des(epoch, model, optim):
    history = []
    train_inputs = []
    train_labels = []
    valid_inputs = []
    valid_labels = []
    
    for inputs, targets in validation_loader:
        valid_inputs.append(inputs)
        valid_labels.append(targets)
        
        
    for inputs_, guesses_ in train_loader:
            train_inputs.append(inputs_)
            train_labels.append(guesses_)

    for ep in range(epoch):       
        pred = model(train_inputs[ep%len(train_inputs)])
        
        err = CEL(pred, train_labels[ep%len(train_labels)])
        model.zero_grad()
        err.backward()
        optim.step()

        if ep % 100 == 0:
            acc = accuracy(pred, train_labels[ep%len(train_labels)])
            acc_val = accuracy( model(valid_inputs[ep%len(valid_inputs)]), valid_labels[ep%len(valid_labels)])
            pred = model(valid_inputs[ep%len(valid_inputs)])
            loss_val = CEL(pred, valid_labels[ep%len(valid_labels)])
            history.append((ep, err.detach().numpy(), acc, loss_val.detach().numpy(), acc_val))
            print(f"Loss at epoch {ep}: {err.item()} accuracy {acc} number of batch {ep%train_loader.batch_size}")
    
    return history
    

optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
history = grad_des(500, model, optimizer)       
plot_history(history)

## Exercice 4